In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [182]:
import pandas as pd
import nltk
from nltk import corpus
from nltk import punkt
from nltk.stem import PorterStemmer, LancasterStemmer,SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics

porter=PorterStemmer()
lancaster=LancasterStemmer()
snowball=SnowballStemmer("english")
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
datafile=r'/gdrive/My Drive/CIS_508/Datasets/Comments.csv'

customer=r'/gdrive/My Drive/CIS_508/Datasets/Customers.csv'

data=pd.read_csv(datafile)
customer_info=pd.read_csv(customer)
data.head()
customer_info.head()

,ID,Sex,Status,Children,Est_Income,Car_Owner,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype,TARGET
0,1,F,S,1,38000.00,N,229.64,24.393333,3,23.56,0.0,206.08,0,CC,Budget,Intnl_discount,Cancelled
1,6,M,M,2,29616.00,N,75.29,49.426667,2,29.78,0.0,45.50,0,CH,FreeLocal,Standard,Current
2,8,M,M,0,19732.80,N,47.25,50.673333,3,24.81,0.0,22.44,0,CC,FreeLocal,Standard,Current
3,11,M,S,2,96.33,N,59.01,56.473333,1,26.13,0.0,32.88,1,CC,Budget,Standard,Current
4,14,F,M,2,52004.80,N,28.14,25.140000,1,5.03,0.0,23.11,0,CH,Budget,Intnl_discount,Cancelled


In [0]:
def display_all(df):
  with pd.option_context("display.max_rows",1000):
    with pd.option_context("display.max_columns",1000):
      return(df)
display_all(data)

,ID,Comments
0,1309,Does not like the way the phone works. It is t...
1,3556,Wanted to know the nearest store location. Wan...
2,2230,Wants to know how to do text messaging. Referr...
3,2312,Asked how to disable call waiting. referred hi...
4,3327,Needs help learning how to use the phone. I su...
...,...,...
2065,3034,Needed help figuring out his bill. I explained...
2066,271,He lost his phone and called to cancel service...
2067,783,Lost the directions to phone and wants another...
2068,1295,Wants to change address.


In [0]:
# use the word tokenmizer for each word of the dataframe row

data["token_words"]=data["Comments"].apply(word_tokenize)
display_all(data)

,ID,Comments,token_words
0,1309,Does not like the way the phone works. It is t...,"[Does, not, like, the, way, the, phone, works,..."
1,3556,Wanted to know the nearest store location. Wan...,"[Wanted, to, know, the, nearest, store, locati..."
2,2230,Wants to know how to do text messaging. Referr...,"[Wants, to, know, how, to, do, text, messaging..."
3,2312,Asked how to disable call waiting. referred hi...,"[Asked, how, to, disable, call, waiting, ., re..."
4,3327,Needs help learning how to use the phone. I su...,"[Needs, help, learning, how, to, use, the, pho..."
...,...,...,...
2065,3034,Needed help figuring out his bill. I explained...,"[Needed, help, figuring, out, his, bill, ., I,..."
2066,271,He lost his phone and called to cancel service...,"[He, lost, his, phone, and, called, to, cancel..."
2067,783,Lost the directions to phone and wants another...,"[Lost, the, directions, to, phone, and, wants,..."
2068,1295,Wants to change address.,"[Wants, to, change, address, .]"


In [0]:
# count the number of words of each row

data["length_before"]=data.apply(lambda row:len(row['token_words']),axis=1)
display_all(data)

,ID,Comments,token_words,length_before
0,1309,Does not like the way the phone works. It is t...,"[Does, not, like, the, way, the, phone, works,...",19
1,3556,Wanted to know the nearest store location. Wan...,"[Wanted, to, know, the, nearest, store, locati...",14
2,2230,Wants to know how to do text messaging. Referr...,"[Wants, to, know, how, to, do, text, messaging...",14
3,2312,Asked how to disable call waiting. referred hi...,"[Asked, how, to, disable, call, waiting, ., re...",13
4,3327,Needs help learning how to use the phone. I su...,"[Needs, help, learning, how, to, use, the, pho...",24
...,...,...,...,...
2065,3034,Needed help figuring out his bill. I explained...,"[Needed, help, figuring, out, his, bill, ., I,...",13
2066,271,He lost his phone and called to cancel service...,"[He, lost, his, phone, and, called, to, cancel...",29
2067,783,Lost the directions to phone and wants another...,"[Lost, the, directions, to, phone, and, wants,...",17
2068,1295,Wants to change address.,"[Wants, to, change, address, .]",5


In [0]:
#using porter stemmer

def stem_sentences_lancaster(sentence):
    #tokens = sentence.split()
    stemmed_tokens = [porter.stem(token) for token in sentence]
    return ' '.join(stemmed_tokens)

data['comments_porter']=data['token_words'].apply(stem_sentences_lancaster)
display_all(data)


,ID,Comments,token_words,length_before,comments_porter
0,1309,Does not like the way the phone works. It is t...,"[Does, not, like, the, way, the, phone, works,...",19,doe not like the way the phone work . It is to...
1,3556,Wanted to know the nearest store location. Wan...,"[Wanted, to, know, the, nearest, store, locati...",14,want to know the nearest store locat . want to...
2,2230,Wants to know how to do text messaging. Referr...,"[Wants, to, know, how, to, do, text, messaging...",14,want to know how to do text messag . refer him...
3,2312,Asked how to disable call waiting. referred hi...,"[Asked, how, to, disable, call, waiting, ., re...",13,ask how to disabl call wait . refer him to web...
4,3327,Needs help learning how to use the phone. I su...,"[Needs, help, learning, how, to, use, the, pho...",24,need help learn how to use the phone . I sugge...
...,...,...,...,...,...
2065,3034,Needed help figuring out his bill. I explained...,"[Needed, help, figuring, out, his, bill, ., I,...",13,need help figur out hi bill . I explain our mi...
2066,271,He lost his phone and called to cancel service...,"[He, lost, his, phone, and, called, to, cancel...",29,He lost hi phone and call to cancel servic . I...
2067,783,Lost the directions to phone and wants another...,"[Lost, the, directions, to, phone, and, wants,...",17,lost the direct to phone and want anoth manual...
2068,1295,Wants to change address.,"[Wants, to, change, address, .]",5,want to chang address .


In [0]:
#using the lancaster stemmer

def stem_sentence_lancaster(sentence):
  
  stemmed_tokens_lancaster=(lancaster.stem(token) for token in sentence)
  return " ".join(stemmed_tokens_lancaster)

data['comments_lancaster']=data['token_words'].apply(stem_sentence_lancaster)
display_all(data)

,ID,Comments,token_words,length_before,comments_porter,comments_lancaster
0,1309,Does not like the way the phone works. It is t...,"[Does, not, like, the, way, the, phone, works,...",19,doe not like the way the phone work . It is to...,doe not lik the way the phon work . it is to d...
1,3556,Wanted to know the nearest store location. Wan...,"[Wanted, to, know, the, nearest, store, locati...",14,want to know the nearest store locat . want to...,want to know the nearest stor loc . want to bu...
2,2230,Wants to know how to do text messaging. Referr...,"[Wants, to, know, how, to, do, text, messaging...",14,want to know how to do text messag . refer him...,want to know how to do text mess . refer him t...
3,2312,Asked how to disable call waiting. referred hi...,"[Asked, how, to, disable, call, waiting, ., re...",13,ask how to disabl call wait . refer him to web...,ask how to dis cal wait . refer him to web sit .
4,3327,Needs help learning how to use the phone. I su...,"[Needs, help, learning, how, to, use, the, pho...",24,need help learn how to use the phone . I sugge...,nee help learn how to us the phon . i suggest ...
...,...,...,...,...,...,...
2065,3034,Needed help figuring out his bill. I explained...,"[Needed, help, figuring, out, his, bill, ., I,...",13,need help figur out hi bill . I explain our mi...,nee help fig out his bil . i explain our minut...
2066,271,He lost his phone and called to cancel service...,"[He, lost, his, phone, and, called, to, cancel...",29,He lost hi phone and call to cancel servic . I...,he lost his phon and cal to cancel serv . i to...
2067,783,Lost the directions to phone and wants another...,"[Lost, the, directions, to, phone, and, wants,...",17,lost the direct to phone and want anoth manual...,lost the direct to phon and want anoth man . i...
2068,1295,Wants to change address.,"[Wants, to, change, address, .]",5,want to chang address .,want to chang address .


In [0]:
#using the snowball stemmer

def stem_sentence_snowball(sentence):
  #tokens=sentence.split()
  stemmed_tokens_snowball=[snowball.stem(token) for token in sentence]
  return " ".join(stemmed_tokens_snowball)

data["comments_snowball"]=data['token_words'].apply(stem_sentence_snowball)
display_all(data)

,ID,Comments,token_words,length_before,comments_porter,comments_lancaster,comments_snowball
0,1309,Does not like the way the phone works. It is t...,"[Does, not, like, the, way, the, phone, works,...",19,doe not like the way the phone work . It is to...,doe not lik the way the phon work . it is to d...,doe not like the way the phone work . it is to...
1,3556,Wanted to know the nearest store location. Wan...,"[Wanted, to, know, the, nearest, store, locati...",14,want to know the nearest store locat . want to...,want to know the nearest stor loc . want to bu...,want to know the nearest store locat . want to...
2,2230,Wants to know how to do text messaging. Referr...,"[Wants, to, know, how, to, do, text, messaging...",14,want to know how to do text messag . refer him...,want to know how to do text mess . refer him t...,want to know how to do text messag . refer him...
3,2312,Asked how to disable call waiting. referred hi...,"[Asked, how, to, disable, call, waiting, ., re...",13,ask how to disabl call wait . refer him to web...,ask how to dis cal wait . refer him to web sit .,ask how to disabl call wait . refer him to web...
4,3327,Needs help learning how to use the phone. I su...,"[Needs, help, learning, how, to, use, the, pho...",24,need help learn how to use the phone . I sugge...,nee help learn how to us the phon . i suggest ...,need help learn how to use the phone . i sugge...
...,...,...,...,...,...,...,...
2065,3034,Needed help figuring out his bill. I explained...,"[Needed, help, figuring, out, his, bill, ., I,...",13,need help figur out hi bill . I explain our mi...,nee help fig out his bil . i explain our minut...,need help figur out his bill . i explain our m...
2066,271,He lost his phone and called to cancel service...,"[He, lost, his, phone, and, called, to, cancel...",29,He lost hi phone and call to cancel servic . I...,he lost his phon and cal to cancel serv . i to...,he lost his phone and call to cancel servic . ...
2067,783,Lost the directions to phone and wants another...,"[Lost, the, directions, to, phone, and, wants,...",17,lost the direct to phone and want anoth manual...,lost the direct to phon and want anoth man . i...,lost the direct to phone and want anoth manual...
2068,1295,Wants to change address.,"[Wants, to, change, address, .]",5,want to chang address .,want to chang address .,want to chang address .


In [0]:
#count the number of words by each type of stemmer

data["length_porter"]=data.apply(lambda row:len(row['comments_porter']),axis=1)

data["length_lancaster"]=data.apply(lambda row:len(row['comments_lancaster']),axis=1)

data["length_snowball"]=data.apply(lambda row:len(row['comments_snowball']),axis=1)


print("number of stemmed words by porter method:",data["length_porter"].sum())
print("nuber of stemmed words by lancaster method:",data["length_lancaster"].sum())
print("nuber of stemmed words by snowball method:",data["length_snowball"].sum())

number of stemmed words by porter method: 166710
nuber of stemmed words by lancaster method: 157299
nuber of stemmed words by snowball method: 167283


In [0]:

#lets create the TF-ID matrix and term document matrix

count_vect=CountVectorizer(stop_words= "english",lowercase=False)

Tfid=TfidfVectorizer(stop_words= "english")


#lets fit to the Tfidvectorizer and find the length of the matrix for each stemmer

#porter stemmer

tdcount_porter=count_vect.fit_transform(data.comments_porter)
print(tdcount_porter.shape)
print(count_vect.get_feature_names())
print("the length for the porter stemmer with countvectorizer:",len(count_vect.vocabulary_))

tdcount_porter_tf=Tfid.fit(data.comments_porter)
print("the length for the porter stemmer with TFIDf:",len(Tfid.vocabulary_))
print(tdcount_porter_tf.get_feature_names())

print('')

print('lancaster stemmer')

tdcount_lancaster_tf=Tfid.fit_transform(data.comments_lancaster)
print("the length for the lancaster stemmer with TFIDf:",len(Tfid.vocabulary_))



tdcount_lancaster_cv=count_vect.fit_transform(data.comments_lancaster)
print("the length for the lancaster stemmer with countvectorizer:",len(count_vect.vocabulary_))
print(count_vect.get_feature_names())
print(tdcount_lancaster_cv.shape)

print("")
print('snowball stemmer')

#snowball stemmer
tdcount_snowball_tf=Tfid.fit_transform(data.comments_snowball)
print("the length for the snowball stemmer with TFIDf:",len(Tfid.vocabulary_))
print(Tfid.get_feature_names())
print(tdcount_snowball_tf.shape)

tdcount_snowball_cv=count_vect.fit_transform(data.comments_snowball)
print("the length for the snowball stemmer with countvectorizer:",len(count_vect.vocabulary_))
print(count_vect.get_feature_names())
print(tdcount_snowball_cv.shape)

#print(tdcount_porter)

td_counts_porter_cv= pd.DataFrame(tdcount_porter.toarray())
td_counts_lancaster_cv= pd.DataFrame(tdcount_lancaster_cv.toarray())
td_counts_snowball_cv= pd.DataFrame(tdcount_snowball_cv.toarray())

(2070, 366)
['3399', '3g', 'As', 'CC', 'He', 'If', 'In', 'Is', 'It', 'We', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constanli', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'd

In [0]:
#TF-IDF Matrix for all the stemmers

transform = TfidfTransformer()
train_tfidf = transform.fit_transform(td_counts_porter_cv)

print(train_tfidf.shape)

TFIDF_porter=pd.DataFrame(train_tfidf.toarray())
display_all(TFIDF_porter.T)

(2070, 366)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.320257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.268523,0.161967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.407251,0.0,0.421944,...,0.326949,0.0,0.0,0.0,0.0,0.0,0.186173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.407251,0.0,0.132355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.407251,0.0,0.0,0.407251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.373340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.429137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.

In [0]:
print("lancaster")

train_tfidf = transform.fit_transform(tdcount_lancaster_cv)

print(train_tfidf.shape)

TFIDF_stemmer=pd.DataFrame(train_tfidf.toarray())
display_all(TFIDF_stemmer.T)

lancaster
(2070, 364)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.275669,0.0,0.0,0.0,0.0,0.267396,0.0,0.0,0.368487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368487,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368487,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
360,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
361,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.435101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
362,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [0]:
print("snowball")

train_tfidf = transform.fit_transform(tdcount_snowball_cv)

print(train_tfidf.shape)

TFIDF_snowball=pd.DataFrame(train_tfidf.toarray())
display_all(TFIDF_snowball.T)

snowball
(2070, 354)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069
0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.00000,0.0,0.0,0.0,0.0,0.580676,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.27568,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.368149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368149,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.368149,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
350,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
351,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
352,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.402625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.436773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [0]:
#lets find our x_train and ytrain from the target column

y_train=customer_info['TARGET']
x_train=customer_info.drop(columns=['TARGET'])

print(x_train.shape)
print(y_train.shape)
display_all(x_train)

(2070, 16)
(2070,)


,ID,Sex,Status,Children,Est_Income,Car_Owner,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,Paymethod,LocalBilltype,LongDistanceBilltype
0,1,F,S,1,38000.00,N,229.64,24.393333,3,23.56,0.00,206.08,0,CC,Budget,Intnl_discount
1,6,M,M,2,29616.00,N,75.29,49.426667,2,29.78,0.00,45.50,0,CH,FreeLocal,Standard
2,8,M,M,0,19732.80,N,47.25,50.673333,3,24.81,0.00,22.44,0,CC,FreeLocal,Standard
3,11,M,S,2,96.33,N,59.01,56.473333,1,26.13,0.00,32.88,1,CC,Budget,Standard
4,14,F,M,2,52004.80,N,28.14,25.140000,1,5.03,0.00,23.11,0,CH,Budget,Intnl_discount
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,F,S,0,78851.30,N,29.04,48.373333,4,0.37,0.00,28.66,0,CC,FreeLocal,Standard
2066,3822,F,S,1,17540.70,Y,36.20,62.786667,1,22.17,0.57,13.45,0,Auto,Budget,Standard
2067,3823,F,M,0,83891.90,Y,74.40,61.020000,4,28.92,0.00,45.47,0,CH,Budget,Standard
2068,3824,F,M,2,28220.80,N,38.95,38.766667,4,26.49,0.00,12.46,0,CC,FreeLocal,Standard


In [0]:
# its time for the feature extraction from the TFIDF using forward search and K best features

#Feature selection using select K best with 75 best features
features_75 = SelectKBest(score_func=chi2, k=75).fit_transform(TFIDF_porter,y_train)
features_75.shape

features_porter_75= pd.DataFrame(features_75)
print(features_porter_75)


       0    1         2    3    4    5   ...   69        70   71   72   73   74
0     0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
1     0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
2     0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
3     0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
4     0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.348322  0.0  0.0  0.0  0.0
...   ...  ...       ...  ...  ...  ...  ...  ...       ...  ...  ...  ...  ...
2065  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.772949  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.000000  0.0  0.0  0.0  ...  0.0  0.000000  0.0  0.0  0.0  0.0

[2070 rows x 75 columns]


In [0]:
#Feature selection using select K best with 90 best features
features_90 = SelectKBest(score_func=chi2, k=90).fit_transform(TFIDF_porter,y_train)
features_90.shape

features_porter_90= pd.DataFrame(features_90)
print(features_porter_90)

       0    1    2    3         4    5   ...        84   85   86   87   88   89
0     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.348322  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...  ...       ...  ...  ...       ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.0  0.772949  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.0  0.000000  0.0  ...  0.000000  0.0  0.0  0.0  0.0  0.0

[2070 rows x 90 columns]


In [0]:
#Feature selection using select K best with 50 best features
features_50 = SelectKBest(score_func=chi2, k=50).fit_transform(TFIDF_porter,y_train)
features_50.shape

features_porter_50= pd.DataFrame(features_50)
print(features_porter_50)


       0         1    2    3    4    5   ...   44   45        46   47   48   49
0     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
1     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
3     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
4     0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.348322  0.0  0.0  0.0
...   ...       ...  ...  ...  ...  ...  ...  ...  ...       ...  ...  ...  ...
2065  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2066  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2067  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2068  0.0  0.772949  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0
2069  0.0  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.000000  0.0  0.0  0.0

[2070 rows x 50 columns]


In [0]:
#Feature selection using select K best with 30 best features
features_30 = SelectKBest(score_func=chi2, k=30).fit_transform(TFIDF_porter,y_train)
features_30.shape

features_porter_30= pd.DataFrame(features_30)
print(features_porter_30)



            0    1    2    3         4         5   ...   24   25   26   27   28   29
0     0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
1     0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
2     0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
3     0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
4     0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
...        ...  ...  ...  ...       ...       ...  ...  ...  ...  ...  ...  ...  ...
2065  0.000000  0.0  0.0  0.0  0.000000  0.432087  ...  0.0  0.0  0.0  0.0  0.0  0.0
2066  0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
2067  0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
2068  0.772949  0.0  0.0  0.0  0.545354  0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0
2069  0.000000  0.0  0.0  0.0  0.000000  0.000000  ...  0.0  0.0 

In [0]:
#Feature selection using select K best with 10 best features
features_10 = SelectKBest(score_func=chi2, k=10).fit_transform(TFIDF_porter,y_train)
features_10.shape

features_porter_10= pd.DataFrame(features_10)
print(features_porter_10)


        0    1    2         3        4    5    6    7    8    9
0     0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...       ...      ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.445599  0.44341  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.000000  0.00000  0.0  0.0  0.0  0.0  0.0

[2070 rows x 10 columns]


In [0]:
# Randomforest tree for 75 features

rf=RandomForestClassifier(n_estimators=90)
rf_select=rf.fit(features_porter_75,y_train)

rf_predict=rf.predict(features_porter_75)

print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predict))
print("Classification Report")
print(classification_report(y_train, rf_predict))

Confusion Matrix:
[[  80  724]
 [  34 1232]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.70      0.10      0.17       804
     Current       0.63      0.97      0.76      1266

    accuracy                           0.63      2070
   macro avg       0.67      0.54      0.47      2070
weighted avg       0.66      0.63      0.54      2070



In [0]:
# Randomforest tree for 50 features

rf=RandomForestClassifier(n_estimators=90)
rf_select=rf.fit(features_porter_50,y_train)

rf_predict=rf.predict(features_porter_50)

print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predict))
print("Classification Report")
print(classification_report(y_train, rf_predict))

Confusion Matrix:
[[  68  736]
 [  24 1242]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.74      0.08      0.15       804
     Current       0.63      0.98      0.77      1266

    accuracy                           0.63      2070
   macro avg       0.68      0.53      0.46      2070
weighted avg       0.67      0.63      0.53      2070



In [0]:
# Randomforest tree for 90 features

rf=RandomForestClassifier(n_estimators=90)
rf_select=rf.fit(features_porter_90,y_train)

rf_predict=rf.predict(features_porter_90)

print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predict))
print("Classification Report")
print(classification_report(y_train, rf_predict))

Confusion Matrix:
[[  92  712]
 [  45 1221]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.67      0.11      0.20       804
     Current       0.63      0.96      0.76      1266

    accuracy                           0.63      2070
   macro avg       0.65      0.54      0.48      2070
weighted avg       0.65      0.63      0.54      2070



In [0]:
# Randomforest tree for 10 features

rf=RandomForestClassifier(n_estimators=90)
rf_select=rf.fit(features_porter_10,y_train)

rf_predict=rf.predict(features_porter_10)

print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predict))
print("Classification Report")
print(classification_report(y_train, rf_predict))

Confusion Matrix:
[[  49  755]
 [  16 1250]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.75      0.06      0.11       804
     Current       0.62      0.99      0.76      1266

    accuracy                           0.63      2070
   macro avg       0.69      0.52      0.44      2070
weighted avg       0.67      0.63      0.51      2070



In [153]:
# Randomforest tree for 90 features

rf=RandomForestClassifier(n_estimators=90)
rf_select=rf.fit(features_porter_90,y_train)

rf_predict=rf.predict(features_porter_90)

print("Confusion Matrix:")
print(confusion_matrix(y_train, rf_predict))
print("Classification Report")
print(classification_report(y_train, rf_predict))

Confusion Matrix:
[[  85  719]
 [  38 1228]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.69      0.11      0.18       804
     Current       0.63      0.97      0.76      1266

    accuracy                           0.63      2070
   macro avg       0.66      0.54      0.47      2070
weighted avg       0.65      0.63      0.54      2070



In [0]:
#Merge files

print(customer_info.shape)


combined=pd.concat([customer_info, features_porter_75], axis=1)
print(combined.shape)
display_all(combined.T)

(2070, 17)
(2070, 92)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069
ID,1,6,8,11,14,17,18,21,22,23,24,29,35,36,37,38,40,42,45,48,52,53,54,60,61,62,63,65,68,71,73,74,75,77,78,79,80,81,83,84,...,3749,3752,3756,3757,3760,3766,3767,3768,3769,3770,3771,3773,3776,3779,3782,3783,3784,3785,3786,3788,3789,3790,3791,3794,3795,3796,3798,3799,3800,3801,3803,3805,3806,3808,3811,3821,3822,3823,3824,3825
Sex,F,M,M,M,F,M,M,M,M,M,F,M,F,F,F,F,F,F,M,F,F,F,F,F,M,F,F,F,F,F,F,M,F,M,F,F,F,F,F,F,...,F,F,F,F,F,M,F,F,F,F,F,M,F,F,F,F,M,M,M,M,F,F,F,M,M,M,M,M,F,M,M,M,M,M,F,F,F,F,F,F
Status,S,M,M,S,M,M,M,M,S,M,M,M,S,S,M,M,S,M,S,S,M,M,M,S,S,M,M,S,S,M,M,M,S,S,M,S,S,M,M,M,...,S,M,M,M,S,S,M,S,S,S,M,M,M,S,M,M,M,S,S,S,S,S,M,M,M,S,S,D,M,M,M,M,S,S,M,S,S,M,M,S
Children,1,2,0,2,2,2,1,0,1,2,2,1,0,1,0,2,0,2,2,1,2,1,2,1,2,2,2,1,2,2,2,1,0,0,0,0,2,2,2,0,...,1,2,2,2,0,1,2,0,1,0,2,2,0,2,0,1,1,1,2,2,0,1,0,0,0,0,0,0,2,1,2,0,1,2,2,0,1,0,2,0
Est_Income,38000,29616,19732.8,96.33,52004.8,53010.8,75004.5,19749.3,57626.9,20078,47902,7545.96,78851.3,17540.7,83891.9,28220.8,28589.1,5237.63,89459.9,13576.5,67388,57063,84166.1,7737.94,100020,45287.6,59613.1,16326.7,9559.78,42031.8,83284.1,39229.1,68427.4,3601.71,72642.6,98716.3,66182.7,84789.3,35976.5,32703.1,...,2800.68,90271.6,64816.6,98217.1,15989.3,84978.1,36647.9,64319.2,73338,433.74,16962.2,10708.3,2031.61,16938.5,55221,8073.11,95448.6,24141.5,63152.6,72084.7,42760.5,49072,27371.8,71472.9,95405.7,95786.8,90321.6,90478.6,56187,28313.1,29616,19732.8,41690.5,96.33,52004.8,78851.3,17540.7,83891.9,28220.8,28589.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,0,0,0,0.348322,0,0,0,0,0,0,0,0,0.348322,0,0,0,0,0,0.299195,0,0,0,0,0,0,0,0,0,0.181885,0.348322,0,0,0,0,0,0,0,0,0,...,0,0,0.348322,0,0.348322,0,0,0.348322,0,0.348322,0,0.348322,0,0,0.348322,0,0,0,0,0,0.150775,0,0,0.348322,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
73,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#one hot encoding of the categorical variables


categorical_columns=combined.columns[combined.dtypes==object].tolist()

one_hot=pd.get_dummies(combined,columns=categorical_columns)

print(one_hot.shape)

(2070, 101)


In [169]:

#modify the 80% of the one hot to the new x_train

x_train_update=one_hot.iloc[414:,:-2]
print(x_train_update.shape)

y_train_update=y_train.iloc[414:,]
print(y_train_update.shape)

display_all(x_train_update)


(1656, 99)
(1656,)


,ID,Children,Est_Income,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,...,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,Sex_F,Sex_M,Status_D,Status_M,Status_S,Car_Owner_N,Car_Owner_Y,Paymethod_Auto,Paymethod_CC,Paymethod_CH,LocalBilltype_Budget,LocalBilltype_FreeLocal,LongDistanceBilltype_Intnl_discount,LongDistanceBilltype_Standard
414,791,2,809.43,2.89,31.913333,4,0.00,0.00,2.89,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,1,0,1,1,0
415,792,1,76529.80,43.89,58.280000,4,18.46,5.36,20.05,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,1,0,0,1,0,1,0,0,1
416,793,2,34577.70,145.75,63.253333,4,29.18,0.00,116.57,3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,0,1,1,0,0,1
417,795,0,66666.00,73.02,67.000000,1,27.56,0.00,45.46,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.432087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.445599,0.0,0.44341,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0,0,1,0,0,1
418,796,2,19008.40,147.74,52.166667,4,10.54,0.00,137.19,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,0,78851.30,29.04,48.373333,4,0.37,0.00,28.66,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.432087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.445599,0.0,0.44341,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,1,0,0,1,0,0,1,0,1
2066,3822,1,17540.70,36.20,62.786667,1,22.17,0.57,13.45,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,1,0,0,1,0,0,1
2067,3823,0,83891.90,74.40,61.020000,4,28.92,0.00,45.47,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,1,1,0,0,1
2068,3824,2,28220.80,38.95,38.766667,4,26.49,0.00,12.46,0,0.0,0.0,0.772949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.545354,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,1,0,0,1,0,1


In [170]:
#Create a Random Forest Classifier


rf_final = rf.fit(x_train_update,y_train_update)
print("Accuracy score (training): {0:.6f}".format(rf.score(x_train_update, y_train_update)))
rf_final_predictions = rf.predict(x_train_update)

print("Confusion Matrix:")
print(confusion_matrix(y_train_update, rf_final_predictions))

print("Classification Report")
print(classification_report(y_train_update, rf_final_predictions))

Accuracy score (training): 1.000000
Confusion Matrix:
[[ 652    0]
 [   0 1004]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       1.00      1.00      1.00       652
     Current       1.00      1.00      1.00      1004

    accuracy                           1.00      1656
   macro avg       1.00      1.00      1.00      1656
weighted avg       1.00      1.00      1.00      1656



In [171]:
#divide the final data set to 80% train and 20% test

x_test=one_hot.iloc[1656:,:-2]
print(x_test.shape)

y_test=y_train.iloc[1656:,]
print(y_test.shape)

(414, 99)
(414,)


In [172]:
#predict the test data

rf_test=rf.fit(x_test,y_test)

rf_final_predictions_test = rf.predict(x_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_final_predictions_test))

print("Classification Report")
print(classification_report(y_test, rf_final_predictions_test))

Confusion Matrix:
[[162   0]
 [  0 252]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       1.00      1.00      1.00       162
     Current       1.00      1.00      1.00       252

    accuracy                           1.00       414
   macro avg       1.00      1.00      1.00       414
weighted avg       1.00      1.00      1.00       414



In [173]:
# Build RF classifier to use in feature selection
rf_wrapper = RandomForestClassifier(n_estimators=100, n_jobs=-1)

# Build step forward feature selection
sfs = sfs(rf_wrapper,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)
  
print(sfs)

# Perform SFFS
sfs_fit = sfs.fit(x_train_update,y_train_update)

SequentialFeatureSelector(clone_estimator=True, cv=5,
                          estimator=RandomForestClassifier(bootstrap=True,
                                                           class_weight=None,
                                                           criterion='gini',
                                                           max_depth=None,
                                                           max_features='auto',
                                                           max_leaf_nodes=None,
                                                           min_impurity_decrease=0.0,
                                                           min_impurity_split=None,
                                                           min_samples_leaf=1,
                                                           min_samples_split=2,
                                                           min_weight_fraction_leaf=0.0,
                                                           n_es

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  99 out of  99 | elapsed:  2.4min finished

[2019-12-14 04:10:50] Features: 1/5 -- score: 0.858757270223437[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  98 out of  98 | elapsed:  2.8min finished

[2019-12-14 04:13:40] Features: 2/5 -- score: 0.8629704646655835[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  97 out of  97 | elapsed:  2.7min finished

[2019-12-14 04:16:24] Features: 3/5 -- score: 0.8696097766512356[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 o

In [175]:
#list of the columns selected as features

features_columns = list(sfs_fit.k_feature_idx_)
print(features_columns)


x_train_wrapper=x_train_update.iloc[:,features_columns]
print(x_train_wrapper.shape)

x_test_wrapper=x_test.iloc[:,features_columns]
print(x_test_wrapper.shape)

display_all(x_train_wrapper)

[2, 4, 5, 8, 86]
(1656, 5)
(414, 5)


,Est_Income,Age,RatePlan,Local,Sex_M
414,809.43,31.913333,4,2.89,0
415,76529.80,58.280000,4,20.05,1
416,34577.70,63.253333,4,116.57,0
417,66666.00,67.000000,1,45.46,0
418,19008.40,52.166667,4,137.19,0
...,...,...,...,...,...
2065,78851.30,48.373333,4,28.66,0
2066,17540.70,62.786667,1,13.45,0
2067,83891.90,61.020000,4,45.47,0
2068,28220.80,38.766667,4,12.46,0


In [176]:
# Build a classifier with selected features

rf_wrapper = RandomForestClassifier(n_estimators=90, random_state=5, max_depth=4)
rf_wrapper.fit(x_train_wrapper, y_train_update)

print("Accuracy score (training): {0:.6f}".format(rf_wrapper.score(x_train_wrapper, y_train_update)))
rf_final_predictions_wrapper = rf_wrapper.predict(x_train_wrapper)

print("Confusion Matrix:")
print(confusion_matrix(y_train_update, rf_final_predictions_wrapper))

print("Classification Report")
print(classification_report(y_train_update, rf_final_predictions_wrapper))
y_train_pred = rf_wrapper.predict(x_test_wrapper)


y_test_pred_wrapper = rf_wrapper.predict(x_test_wrapper)

print("Accuracy score (testing): {0:.6f}".format(rf_wrapper.score(x_train_wrapper, y_train_update)))

print("Confusion Matrix:")
print(confusion_matrix(y_test_pred_wrapper,y_test_pred_wrapper ))

print("Classification Report")
print(classification_report(y_test, y_test_pred_wrapper))
y_train_pred = rf_wrapper.predict(x_test_wrapper)

Accuracy score (training): 0.836957
Confusion Matrix:
[[457 195]
 [ 75 929]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.86      0.70      0.77       652
     Current       0.83      0.93      0.87      1004

    accuracy                           0.84      1656
   macro avg       0.84      0.81      0.82      1656
weighted avg       0.84      0.84      0.83      1656

Accuracy score (testing): 0.836957
Confusion Matrix:
[[142   0]
 [  0 272]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.84      0.73      0.78       162
     Current       0.84      0.91      0.87       252

    accuracy                           0.84       414
   macro avg       0.84      0.82      0.83       414
weighted avg       0.84      0.84      0.84       414



In [0]:
#Merge files for 50 features

print(customer_info.shape)


combined_dt=pd.concat([customer_info, features_porter_50], axis=1)
print(combined_dt.shape)
display_all(combined_dt.T)

(2070, 17)
(2070, 67)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069
ID,1,6,8,11,14,17,18,21,22,23,24,29,35,36,37,38,40,42,45,48,52,53,54,60,61,62,63,65,68,71,73,74,75,77,78,79,80,81,83,84,...,3749,3752,3756,3757,3760,3766,3767,3768,3769,3770,3771,3773,3776,3779,3782,3783,3784,3785,3786,3788,3789,3790,3791,3794,3795,3796,3798,3799,3800,3801,3803,3805,3806,3808,3811,3821,3822,3823,3824,3825
Sex,F,M,M,M,F,M,M,M,M,M,F,M,F,F,F,F,F,F,M,F,F,F,F,F,M,F,F,F,F,F,F,M,F,M,F,F,F,F,F,F,...,F,F,F,F,F,M,F,F,F,F,F,M,F,F,F,F,M,M,M,M,F,F,F,M,M,M,M,M,F,M,M,M,M,M,F,F,F,F,F,F
Status,S,M,M,S,M,M,M,M,S,M,M,M,S,S,M,M,S,M,S,S,M,M,M,S,S,M,M,S,S,M,M,M,S,S,M,S,S,M,M,M,...,S,M,M,M,S,S,M,S,S,S,M,M,M,S,M,M,M,S,S,S,S,S,M,M,M,S,S,D,M,M,M,M,S,S,M,S,S,M,M,S
Children,1,2,0,2,2,2,1,0,1,2,2,1,0,1,0,2,0,2,2,1,2,1,2,1,2,2,2,1,2,2,2,1,0,0,0,0,2,2,2,0,...,1,2,2,2,0,1,2,0,1,0,2,2,0,2,0,1,1,1,2,2,0,1,0,0,0,0,0,0,2,1,2,0,1,2,2,0,1,0,2,0
Est_Income,38000,29616,19732.8,96.33,52004.8,53010.8,75004.5,19749.3,57626.9,20078,47902,7545.96,78851.3,17540.7,83891.9,28220.8,28589.1,5237.63,89459.9,13576.5,67388,57063,84166.1,7737.94,100020,45287.6,59613.1,16326.7,9559.78,42031.8,83284.1,39229.1,68427.4,3601.71,72642.6,98716.3,66182.7,84789.3,35976.5,32703.1,...,2800.68,90271.6,64816.6,98217.1,15989.3,84978.1,36647.9,64319.2,73338,433.74,16962.2,10708.3,2031.61,16938.5,55221,8073.11,95448.6,24141.5,63152.6,72084.7,42760.5,49072,27371.8,71472.9,95405.7,95786.8,90321.6,90478.6,56187,28313.1,29616,19732.8,41690.5,96.33,52004.8,78851.3,17540.7,83891.9,28220.8,28589.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,0,0,0,0,0.348322,0,0,0,0,0,0,0,0,0.348322,0,0,0,0,0,0.299195,0,0,0,0,0,0,0,0,0,0.181885,0.348322,0,0,0,0,0,0,0,0,0,...,0,0,0.348322,0,0.348322,0,0,0.348322,0,0.348322,0,0.348322,0,0,0.348322,0,0,0,0,0,0.150775,0,0,0.348322,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#one hot encoding of the categorical variables


categorical_columns=combined_dt.columns[combined_dt.dtypes==object].tolist()

one_hot_dt=pd.get_dummies(combined_dt,columns=categorical_columns)

print(one_hot_dt.shape)

(2070, 76)


In [189]:
#modify the 80% of the one hot to the new x_train

x_train_dt=one_hot_dt.iloc[414:,:-2]
print(x_train_dt.shape)

y_train_dt=y_train.iloc[414:,]
print(y_train_dt.shape)


x_test_dt=one_hot_dt.iloc[1656:,:-2]
print(x_test_dt.shape)

y_test=y_train.iloc[1656:,]
print(y_test.shape)

display_all(x_train_dt)

(1656, 74)
(1656,)
(414, 74)
(414,)


,ID,Children,Est_Income,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,Sex_F,Sex_M,Status_D,Status_M,Status_S,Car_Owner_N,Car_Owner_Y,Paymethod_Auto,Paymethod_CC,Paymethod_CH,LocalBilltype_Budget,LocalBilltype_FreeLocal,LongDistanceBilltype_Intnl_discount,LongDistanceBilltype_Standard
414,791,2,809.43,2.89,31.913333,4,0.00,0.00,2.89,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,1,0,1,1,0
415,792,1,76529.80,43.89,58.280000,4,18.46,5.36,20.05,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,1,0,0,1,0,1,0,0,1
416,793,2,34577.70,145.75,63.253333,4,29.18,0.00,116.57,3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,0,1,1,0,0,1
417,795,0,66666.00,73.02,67.000000,1,27.56,0.00,45.46,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.432087,0.0,0.0,0.0,0.0,0.0,0.445599,0.0,0.44341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.406057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0,0,1,0,0,1
418,796,2,19008.40,147.74,52.166667,4,10.54,0.00,137.19,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,0,78851.30,29.04,48.373333,4,0.37,0.00,28.66,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.432087,0.0,0.0,0.0,0.0,0.0,0.445599,0.0,0.44341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.406057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,1,0,0,1,0,0,1,0,1
2066,3822,1,17540.70,36.20,62.786667,1,22.17,0.57,13.45,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,1,0,0,1,0,0,1
2067,3823,0,83891.90,74.40,61.020000,4,28.92,0.00,45.47,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,1,1,0,0,1
2068,3824,2,28220.80,38.95,38.766667,4,26.49,0.00,12.46,0,0.0,0.772949,0.0,0.0,0.0,0.0,0.545354,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,1,0,0,1,0,1


In [211]:
#Create a Decision Tree  Classifier

dt_tree=DecisionTreeClassifier()

dt_tree_final = dt_tree.fit(x_train_dt,y_train_dt)
print("Accuracy score (training): {0:.6f}".format(dt_tree_final.score(x_train_dt, y_train_dt)))
dt_final_predictions = dt_tree_final.predict(x_train_dt)

print("Confusion Matrix:")
print(confusion_matrix(y_train_dt, dt_final_predictions))

print("Classification Report")
print(classification_report(y_train_dt, dt_final_predictions))

Accuracy score (training): 1.000000
Confusion Matrix:
[[ 652    0]
 [   0 1004]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       1.00      1.00      1.00       652
     Current       1.00      1.00      1.00      1004

    accuracy                           1.00      1656
   macro avg       1.00      1.00      1.00      1656
weighted avg       1.00      1.00      1.00      1656



In [179]:
#predict the test data

dt_test=dt_tree.fit(x_test_dt,y_test)

dt_final_predictions_test = dt_test.predict(x_test_dt)

print("Confusion Matrix:")
print(confusion_matrix(y_test, dt_final_predictions_test))

print("Classification Report")
print(classification_report(y_test, dt_final_predictions_test))

Confusion Matrix:
[[162   0]
 [  0 252]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       1.00      1.00      1.00       162
     Current       1.00      1.00      1.00       252

    accuracy                           1.00       414
   macro avg       1.00      1.00      1.00       414
weighted avg       1.00      1.00      1.00       414



In [0]:
#decision tree with features_porter_

# Build RF classifier to use in feature selection
dt_wrapper = DecisionTreeClassifier()

# Build step forward feature selection
sfs_dt = sfs(dt_wrapper,
           k_features=3,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

In [184]:
print(sfs_dt)


# Perform SFFS
sfs_dt_final = sfs_dt.fit(x_train_dt,y_train_dt)

SequentialFeatureSelector(clone_estimator=True, cv=5,
                          estimator=DecisionTreeClassifier(class_weight=None,
                                                           criterion='gini',
                                                           max_depth=None,
                                                           max_features=None,
                                                           max_leaf_nodes=None,
                                                           min_impurity_decrease=0.0,
                                                           min_impurity_split=None,
                                                           min_samples_leaf=1,
                                                           min_samples_split=2,
                                                           min_weight_fraction_leaf=0.0,
                                                           presort=False,
                                                           random_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:    1.4s finished

[2019-12-14 04:39:43] Features: 1/3 -- score: 0.8599602805619622[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  73 out of  73 | elapsed:    1.7s finished

[2019-12-14 04:39:45] Features: 2/3 -- score: 0.8659861527957476[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    1.7s finished

[2019-12-14 04:39:47] Features: 3/3 -- score: 0.8690055029235315

In [191]:
#list of the columns selected as features from decision tree

features_columns_dt = list(sfs_dt_final.k_feature_idx_)
print(features_columns_dt)


[3, 8, 60]


In [193]:
x_train_wrapper_dt=one_hot_dt.iloc[414:,features_columns_dt]
print(x_train_wrapper_dt.shape)

x_test_wrapper_dt=one_hot_dt.iloc[1656:,features_columns_dt]
print(x_test_wrapper_dt.shape)

display_all(x_train_wrapper_dt)



(1656, 3)
(414, 3)


,Usage,Local,Sex_F
414,2.89,2.89,1
415,43.89,20.05,0
416,145.75,116.57,1
417,73.02,45.46,1
418,147.74,137.19,1
...,...,...,...
2065,29.04,28.66,1
2066,36.20,13.45,1
2067,74.40,45.47,1
2068,38.95,12.46,1


In [194]:
# Build a classifier with selected features

dt_rapper = DecisionTreeClassifier(max_depth=4)
dt_rapper.fit(x_train_wrapper_dt, y_train_dt)

print("Accuracy score (training): {0:.6f}".format(dt_rapper.score(x_train_wrapper_dt, y_train_dt)))
dt_final_predictions_wrapper = dt_rapper.predict(x_train_wrapper_dt)

print("Confusion Matrix:")
print(confusion_matrix(y_train_dt, dt_final_predictions_wrapper))

print("Classification Report")
print(classification_report(y_train_dt, dt_final_predictions_wrapper))
y_train_pred = dt_rapper.predict(x_test_wrapper_dt)


y_test_pred_wrapper = dt_rapper.predict(x_test_wrapper_dt)

print("Accuracy score (testing): {0:.6f}".format(dt_rapper.score(x_train_wrapper_dt, y_train_dt)))

print("Confusion Matrix:")
print(confusion_matrix(y_test_pred_wrapper,y_test_pred_wrapper ))

print("Classification Report")
print(classification_report(y_test, y_test_pred_wrapper))
y_train_pred = dt_rapper.predict(x_test_wrapper_dt)

Accuracy score (training): 0.691425
Confusion Matrix:
[[311 341]
 [170 834]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.65      0.48      0.55       652
     Current       0.71      0.83      0.77      1004

    accuracy                           0.69      1656
   macro avg       0.68      0.65      0.66      1656
weighted avg       0.68      0.69      0.68      1656

Accuracy score (testing): 0.691425
Confusion Matrix:
[[137   0]
 [  0 277]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.68      0.57      0.62       162
     Current       0.75      0.83      0.79       252

    accuracy                           0.73       414
   macro avg       0.71      0.70      0.70       414
weighted avg       0.72      0.73      0.72       414



In [196]:
#Merge files with the actual TFIDF vector

print(customer_info.shape)


combined_mix=pd.concat([customer_info, TFIDF_porter], axis=1)
print(combined_mix.shape)
display_all(combined_mix.T)

(2070, 17)
(2070, 383)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069
ID,1,6,8,11,14,17,18,21,22,23,24,29,35,36,37,38,40,42,45,48,52,53,54,60,61,62,63,65,68,71,73,74,75,77,78,79,80,81,83,84,...,3749,3752,3756,3757,3760,3766,3767,3768,3769,3770,3771,3773,3776,3779,3782,3783,3784,3785,3786,3788,3789,3790,3791,3794,3795,3796,3798,3799,3800,3801,3803,3805,3806,3808,3811,3821,3822,3823,3824,3825
Sex,F,M,M,M,F,M,M,M,M,M,F,M,F,F,F,F,F,F,M,F,F,F,F,F,M,F,F,F,F,F,F,M,F,M,F,F,F,F,F,F,...,F,F,F,F,F,M,F,F,F,F,F,M,F,F,F,F,M,M,M,M,F,F,F,M,M,M,M,M,F,M,M,M,M,M,F,F,F,F,F,F
Status,S,M,M,S,M,M,M,M,S,M,M,M,S,S,M,M,S,M,S,S,M,M,M,S,S,M,M,S,S,M,M,M,S,S,M,S,S,M,M,M,...,S,M,M,M,S,S,M,S,S,S,M,M,M,S,M,M,M,S,S,S,S,S,M,M,M,S,S,D,M,M,M,M,S,S,M,S,S,M,M,S
Children,1,2,0,2,2,2,1,0,1,2,2,1,0,1,0,2,0,2,2,1,2,1,2,1,2,2,2,1,2,2,2,1,0,0,0,0,2,2,2,0,...,1,2,2,2,0,1,2,0,1,0,2,2,0,2,0,1,1,1,2,2,0,1,0,0,0,0,0,0,2,1,2,0,1,2,2,0,1,0,2,0
Est_Income,38000,29616,19732.8,96.33,52004.8,53010.8,75004.5,19749.3,57626.9,20078,47902,7545.96,78851.3,17540.7,83891.9,28220.8,28589.1,5237.63,89459.9,13576.5,67388,57063,84166.1,7737.94,100020,45287.6,59613.1,16326.7,9559.78,42031.8,83284.1,39229.1,68427.4,3601.71,72642.6,98716.3,66182.7,84789.3,35976.5,32703.1,...,2800.68,90271.6,64816.6,98217.1,15989.3,84978.1,36647.9,64319.2,73338,433.74,16962.2,10708.3,2031.61,16938.5,55221,8073.11,95448.6,24141.5,63152.6,72084.7,42760.5,49072,27371.8,71472.9,95405.7,95786.8,90321.6,90478.6,56187,28313.1,29616,19732.8,41690.5,96.33,52004.8,78851.3,17540.7,83891.9,28220.8,28589.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
362,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
363,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
364,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.37334,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.429137,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [197]:
#one hot encoding of the categorical variables


categorical_columns=combined_mix.columns[combined_mix.dtypes==object].tolist()

one_hot_mix=pd.get_dummies(combined_mix,columns=categorical_columns)

print(one_hot_mix.shape)

(2070, 392)


In [198]:
#modify the 80% of the one hot to the new x_train

x_train_mix=one_hot_dt.iloc[414:,:-2]
print(x_train_mix.shape)

y_train_mix=y_train.iloc[414:,]
print(y_train_mix.shape)


x_test_mix=one_hot_mix.iloc[1656:,:-2]
print(x_test_mix.shape)

y_test=y_train.iloc[1656:,]
print(y_test.shape)

display_all(x_train_mix)

(1656, 74)
(1656,)
(414, 390)
(414,)


,ID,Children,Est_Income,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,Sex_F,Sex_M,Status_D,Status_M,Status_S,Car_Owner_N,Car_Owner_Y,Paymethod_Auto,Paymethod_CC,Paymethod_CH,LocalBilltype_Budget,LocalBilltype_FreeLocal,LongDistanceBilltype_Intnl_discount,LongDistanceBilltype_Standard
414,791,2,809.43,2.89,31.913333,4,0.00,0.00,2.89,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,1,0,1,1,0
415,792,1,76529.80,43.89,58.280000,4,18.46,5.36,20.05,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,1,0,0,1,0,1,0,0,1
416,793,2,34577.70,145.75,63.253333,4,29.18,0.00,116.57,3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,0,1,1,0,0,1
417,795,0,66666.00,73.02,67.000000,1,27.56,0.00,45.46,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.432087,0.0,0.0,0.0,0.0,0.0,0.445599,0.0,0.44341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.406057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0,0,1,0,0,1
418,796,2,19008.40,147.74,52.166667,4,10.54,0.00,137.19,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,3821,0,78851.30,29.04,48.373333,4,0.37,0.00,28.66,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.432087,0.0,0.0,0.0,0.0,0.0,0.445599,0.0,0.44341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.406057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,1,0,0,1,0,0,1,0,1
2066,3822,1,17540.70,36.20,62.786667,1,22.17,0.57,13.45,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,1,0,0,1,0,0,1
2067,3823,0,83891.90,74.40,61.020000,4,28.92,0.00,45.47,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,1,1,0,0,1
2068,3824,2,28220.80,38.95,38.766667,4,26.49,0.00,12.46,0,0.0,0.772949,0.0,0.0,0.0,0.0,0.545354,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,1,0,0,1,0,1


In [208]:
#Create a Gradient Boosting Classifier

gb=GradientBoostingClassifier()

gb_final = gb.fit(x_train_mix,y_train_mix)
print("Accuracy score (training): {0:.6f}".format(gb_final.score(x_train_mix, y_train_mix)))
gb_final_predictions = gb_final.predict(x_train_mix)

print("Confusion Matrix:")
print(confusion_matrix(y_train_mix, gb_final_predictions))

print("Classification Report")
print(classification_report(y_train_mix, gb_final_predictions))

Accuracy score (training): 0.911836
Confusion Matrix:
[[554  98]
 [ 48 956]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.92      0.85      0.88       652
     Current       0.91      0.95      0.93      1004

    accuracy                           0.91      1656
   macro avg       0.91      0.90      0.91      1656
weighted avg       0.91      0.91      0.91      1656



In [202]:
#predict the test data

gb_test=gb.fit(x_test_mix,y_test)

gb_final_predictions_test = gb_test.predict(x_test_mix)

print("Confusion Matrix:")
print(confusion_matrix(y_test, gb_final_predictions_test))

print("Classification Report")
print(classification_report(y_test, gb_final_predictions_test))

Confusion Matrix:
[[155   7]
 [  3 249]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.98      0.96      0.97       162
     Current       0.97      0.99      0.98       252

    accuracy                           0.98       414
   macro avg       0.98      0.97      0.97       414
weighted avg       0.98      0.98      0.98       414



In [203]:
#decision tree with features_porter_

# Build GB classifier to use in feature selection
gb_wrapper = GradientBoostingClassifier()

# Build step forward feature selection
sfs_gb = sfs(gb_wrapper,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

print(sfs_gb)


# Perform SFFS
sfs_gb_final = sfs_gb.fit(x_train_mix,y_train_mix)

SequentialFeatureSelector(clone_estimator=True, cv=5,
                          estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                               init=None,
                                                               learning_rate=0.1,
                                                               loss='deviance',
                                                               max_depth=3,
                                                               max_features=None,
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                                               min_samples_leaf=1,
                                                               min_samples_split=2,
                                  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  74 out of  74 | elapsed:   18.7s finished

[2019-12-14 05:01:22] Features: 1/5 -- score: 0.7984033138872435[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  73 out of  73 | elapsed:   28.0s finished

[2019-12-14 05:01:50] Features: 2/5 -- score: 0.8243962612327916[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   30.3s finished

[2019-12-14 05:02:21] Features: 3/5 -- score: 0.8388741342739813[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

In [205]:
#list of the columns selected as features from decision tree

features_columns_gb = list(sfs_gb_final.k_feature_idx_)
print(features_columns_gb)


[2, 3, 4, 6, 60]


In [207]:
x_train_wrapper_gb=one_hot_mix.iloc[414:,features_columns_gb]
print(x_train_wrapper_gb.shape)

x_test_wrapper_gb=one_hot_mix.iloc[1656:,features_columns_gb]
print(x_test_wrapper_gb.shape)

display_all(x_train_wrapper_gb)


(1656, 5)
(414, 5)


,Est_Income,Usage,Age,LongDistance,50
414,809.43,2.89,31.913333,0.00,0.0
415,76529.80,43.89,58.280000,18.46,0.0
416,34577.70,145.75,63.253333,29.18,0.0
417,66666.00,73.02,67.000000,27.56,0.0
418,19008.40,147.74,52.166667,10.54,0.0
...,...,...,...,...,...
2065,78851.30,29.04,48.373333,0.37,0.0
2066,17540.70,36.20,62.786667,22.17,0.0
2067,83891.90,74.40,61.020000,28.92,0.0
2068,28220.80,38.95,38.766667,26.49,0.0


In [210]:
# Build a classifier with selected features

gb_rapper = GradientBoostingClassifier()
gb_rapper.fit(x_train_wrapper_gb, y_train_mix)

print("Accuracy score (training): {0:.6f}".format(gb_rapper.score(x_train_wrapper_gb, y_train_mix)))
gb_final_predictions_wrapper = gb_rapper.predict(x_train_wrapper_gb)

print("Confusion Matrix:")
print(confusion_matrix(y_train_mix, gb_final_predictions_wrapper))

print("Classification Report")
print(classification_report(y_train_mix, gb_final_predictions_wrapper))
y_train_pred = gb_rapper.predict(x_test_wrapper_gb)


y_test_pred_wrapper = gb_rapper.predict(x_test_wrapper_gb)

print("Accuracy score (testing): {0:.6f}".format(gb_rapper.score(x_train_wrapper_gb, y_train_mix)))

print("Confusion Matrix:")
print(confusion_matrix(y_test_pred_wrapper,y_test_pred_wrapper ))

print("Classification Report")
print(classification_report(y_test, y_test_pred_wrapper))
y_train_pred = gb_rapper.predict(x_test_wrapper_gb)

Accuracy score (training): 0.877415
Confusion Matrix:
[[505 147]
 [ 56 948]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.90      0.77      0.83       652
     Current       0.87      0.94      0.90      1004

    accuracy                           0.88      1656
   macro avg       0.88      0.86      0.87      1656
weighted avg       0.88      0.88      0.88      1656

Accuracy score (testing): 0.877415
Confusion Matrix:
[[152   0]
 [  0 262]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.90      0.85      0.87       162
     Current       0.90      0.94      0.92       252

    accuracy                           0.90       414
   macro avg       0.90      0.89      0.90       414
weighted avg       0.90      0.90      0.90       414

